In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MaxAbsScaler
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
!pip install mealpy

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [4]:
url_file = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat'
columns = ['frequency',
           'angle_of_attack', 
           'chord_length', 
           'free_stream_velocity',
           'suction_side_displacement_thickness',
           'scaled_sound_pressure_level']

features = ['frequency','angle_of_attack', 
            'chord_length', 
            'free_stream_velocity',
            'suction_side_displacement_thickness']

df = pd.read_csv(url_file, sep='\t', header=None, names=columns)

In [5]:
df.head()

,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [6]:
df.tail()

,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224
1502,6300,15.6,0.1016,39.6,0.052849,104.204


In [7]:
df.describe()

,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
count,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000
mean,2886.380572,6.782302,0.136548,50.860745,0.011140,124.835943
std,3152.573137,5.918128,0.093541,15.572784,0.013150,6.898657
min,200.000000,0.000000,0.025400,31.700000,0.000401,103.380000
25%,800.000000,2.000000,0.050800,39.600000,0.002535,120.191000
50%,1600.000000,5.400000,0.101600,39.600000,0.004957,125.721000
75%,4000.000000,9.900000,0.228600,71.300000,0.015576,129.995500
max,20000.000000,22.200000,0.304800,71.300000,0.058411,140.987000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   frequency                            1503 non-null   int64  
 1   angle_of_attack                      1503 non-null   float64
 2   chord_length                         1503 non-null   float64
 3   free_stream_velocity                 1503 non-null   float64
 4   suction_side_displacement_thickness  1503 non-null   float64
 5   scaled_sound_pressure_level          1503 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 70.6 KB


In [9]:
def preprocess_input(df):
    
    train_data = df.copy()
    train_set = train_data.drop("scaled_sound_pressure_level", axis = 1)
    train_labels = train_data["scaled_sound_pressure_level"].copy()
    
    train_set = QuantileTransformer(random_state=0).fit_transform(train_set)
    train_set = MaxAbsScaler().fit_transform(train_set)
    train_set = StandardScaler().fit_transform(train_set)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(train_set, train_labels, test_size=0.2)
    
    return X_train, X_test, y_train, y_test

In [10]:
X_train, X_test, y_train, y_test = preprocess_input(df)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1202, 5)
(301, 5)
(1202,)
(301,)


In [12]:
model = Sequential()

model.add(Input(X_train.shape[1],))

model.add(Dense(32, activation = 'relu'))

model.add(Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                192       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 225 (900.00 Byte)
Trainable params: 225 (900.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.compile(loss = 'mean_squared_error', metrics = tf.keras.metrics.RootMeanSquaredError())

In [14]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes

In [15]:
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape) + index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [16]:
shape = get_shape(model)
num_solutions = 15
dimensions = 225

In [17]:
def evaluate_nn(W):
    model.set_weights(set_shape(W,shape))
    score = model.evaluate(X_train, y_train, verbose = 0)
    return score[1]

In [18]:
from mealpy import ALO, FloatVar
lb = [-10 for i in range(dimensions)]
ub = [10 for i in range(dimensions)]

In [19]:
problem_dict = {
    "bounds": FloatVar(lb = lb, ub = ub, name="NeuralNetwork"),
    "minmax": "min",
    "obj_func": evaluate_nn
}

In [20]:
ALO_optimizer = ALO.OriginalALO(epoch=1000, pop_size=15)

In [21]:
g_best = ALO_optimizer.solve(problem_dict)

2024/02/28 01:29:28 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: Solving single objective optimization problem.
2024/02/28 01:29:35 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 1, Current best: 93.03325653076172, Global best: 93.03325653076172, Runtime: 3.25692 seconds
2024/02/28 01:29:38 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 2, Current best: 93.03325653076172, Global best: 93.03325653076172, Runtime: 3.50131 seconds
2024/02/28 01:29:41 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 3, Current best: 93.03325653076172, Global best: 93.03325653076172, Runtime: 3.37244 seconds
2024/02/28 01:29:45 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 4, Current best: 93.03325653076172, Global best: 93.03325653076172, Runtime: 3.38634 seconds
2024/02/28 01:29:48 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 5, Current best: 43.59540557861328, Global best: 43.59540557861328, Runtime

2024/02/28 01:32:05 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 46, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 3.13472 seconds
2024/02/28 01:32:07 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 47, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 2.53602 seconds
2024/02/28 01:32:10 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 48, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 2.54202 seconds
2024/02/28 01:32:13 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 49, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 3.53342 seconds
2024/02/28 01:32:17 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 50, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 3.20964 seconds
2024/02/28 01:32:19 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P,

2024/02/28 01:34:40 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 91, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 2.90548 seconds
2024/02/28 01:34:43 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 92, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 2.82234 seconds
2024/02/28 01:34:46 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 93, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 3.00691 seconds
2024/02/28 01:34:49 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 94, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 2.66112 seconds
2024/02/28 01:34:51 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 95, Current best: 43.304908752441406, Global best: 43.304908752441406, Runtime: 2.68396 seconds
2024/02/28 01:34:54 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P,

2024/02/28 01:37:06 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 136, Current best: 25.816017150878906, Global best: 25.816017150878906, Runtime: 3.24760 seconds
2024/02/28 01:37:09 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 137, Current best: 25.816017150878906, Global best: 25.816017150878906, Runtime: 3.68116 seconds
2024/02/28 01:37:13 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 138, Current best: 25.516183853149414, Global best: 25.516183853149414, Runtime: 3.51621 seconds
2024/02/28 01:37:16 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 139, Current best: 25.516183853149414, Global best: 25.516183853149414, Runtime: 3.43996 seconds
2024/02/28 01:37:20 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 140, Current best: 25.408178329467773, Global best: 25.408178329467773, Runtime: 3.30710 seconds
2024/02/28 01:37:22 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Proble

2024/02/28 01:39:25 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 181, Current best: 17.19828224182129, Global best: 17.19828224182129, Runtime: 3.19013 seconds
2024/02/28 01:39:28 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 182, Current best: 17.08428382873535, Global best: 17.08428382873535, Runtime: 3.35530 seconds
2024/02/28 01:39:31 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 183, Current best: 17.08428382873535, Global best: 17.08428382873535, Runtime: 3.14660 seconds
2024/02/28 01:39:35 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 184, Current best: 16.987167358398438, Global best: 16.987167358398438, Runtime: 3.54800 seconds
2024/02/28 01:39:39 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 185, Current best: 16.951183319091797, Global best: 16.951183319091797, Runtime: 3.84350 seconds
2024/02/28 01:39:42 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, 

...

2024/02/28 02:16:15 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 811, Current best: 8.74339485168457, Global best: 8.74339485168457, Runtime: 2.24637 seconds
2024/02/28 02:16:17 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 812, Current best: 8.742805480957031, Global best: 8.742805480957031, Runtime: 2.42071 seconds
2024/02/28 02:16:20 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 813, Current best: 8.74238109588623, Global best: 8.74238109588623, Runtime: 2.35004 seconds
2024/02/28 02:16:22 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 814, Current best: 8.741880416870117, Global best: 8.741880416870117, Runtime: 2.20620 seconds
2024/02/28 02:16:24 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 815, Current best: 8.741433143615723, Global best: 8.741433143615723, Runtime: 2.51593 seconds
2024/02/28 02:16:27 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 8

2024/02/28 02:18:51 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 856, Current best: 8.723690032958984, Global best: 8.723690032958984, Runtime: 2.23743 seconds
2024/02/28 02:18:53 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 857, Current best: 8.723514556884766, Global best: 8.723514556884766, Runtime: 2.49947 seconds
2024/02/28 02:18:56 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 858, Current best: 8.72292423248291, Global best: 8.72292423248291, Runtime: 2.44455 seconds
2024/02/28 02:18:58 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 859, Current best: 8.722593307495117, Global best: 8.722593307495117, Runtime: 2.25838 seconds
2024/02/28 02:19:00 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 860, Current best: 8.721712112426758, Global best: 8.721712112426758, Runtime: 2.23608 seconds
2024/02/28 02:19:02 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch:

2024/02/28 02:20:58 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 901, Current best: 8.70285415649414, Global best: 8.70285415649414, Runtime: 3.10759 seconds
2024/02/28 02:21:02 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 902, Current best: 8.702800750732422, Global best: 8.702800750732422, Runtime: 3.17307 seconds
2024/02/28 02:21:05 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 903, Current best: 8.702780723571777, Global best: 8.702780723571777, Runtime: 3.34267 seconds
2024/02/28 02:21:09 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 904, Current best: 8.702714920043945, Global best: 8.702714920043945, Runtime: 3.84466 seconds
2024/02/28 02:21:13 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 905, Current best: 8.702690124511719, Global best: 8.702690124511719, Runtime: 4.06767 seconds
2024/02/28 02:21:17 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch:

2024/02/28 02:23:11 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 946, Current best: 8.700509071350098, Global best: 8.700509071350098, Runtime: 2.15907 seconds
2024/02/28 02:23:14 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 947, Current best: 8.700399398803711, Global best: 8.700399398803711, Runtime: 2.36877 seconds
2024/02/28 02:23:16 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 948, Current best: 8.700338363647461, Global best: 8.700338363647461, Runtime: 2.60093 seconds
2024/02/28 02:23:19 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 949, Current best: 8.700303077697754, Global best: 8.700303077697754, Runtime: 3.00266 seconds
2024/02/28 02:23:22 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 950, Current best: 8.700264930725098, Global best: 8.700264930725098, Runtime: 2.86300 seconds
2024/02/28 02:23:25 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoc

2024/02/28 02:25:03 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 991, Current best: 8.700063705444336, Global best: 8.700063705444336, Runtime: 2.43604 seconds
2024/02/28 02:25:06 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 992, Current best: 8.700053215026855, Global best: 8.700053215026855, Runtime: 2.58631 seconds
2024/02/28 02:25:08 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 993, Current best: 8.700045585632324, Global best: 8.700045585632324, Runtime: 2.53591 seconds
2024/02/28 02:25:10 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 994, Current best: 8.700044631958008, Global best: 8.700044631958008, Runtime: 2.30840 seconds
2024/02/28 02:25:13 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch: 995, Current best: 8.70003604888916, Global best: 8.70003604888916, Runtime: 2.23924 seconds
2024/02/28 02:25:15 PM, INFO, mealpy.swarm_based.ALO.OriginalALO: >>>Problem: P, Epoch:

In [22]:
solution = g_best.solution
model.set_weights(set_shape(solution, shape))

In [23]:
# RMSE
score = model.evaluate(X_train, y_train)
train_rmse, train_mse = score[1], score[0]

score = model.evaluate(X_test, y_test)
test_rmse, test_mse = score[1], score[0]

print("Train RMSE = ", train_rmse)
print("Test RMSE = ", test_rmse)
print("---------------------------")

#MSE
print("Train MSE = ", train_mse)
print("Test MSE = ", test_mse)
print("---------------------------")

#R2 Score
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_r2_score = r2_score(y_train, y_train_pred)
test_r2_score = r2_score(y_test, y_test_pred)

print("Train R2 = ", train_r2_score)
print("Test R2 = ", test_r2_score)
print("-----------------------------")

# MAE
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
print("Train MAE = ", train_mae)
print("Test MAE = ", test_mae)

10/10 [==============================] - 0s 2ms/step - loss: 80.5227 - root_mean_squared_error: 8.9734
Train RMSE =  8.7000093460083
Test RMSE =  8.973445892333984
---------------------------
Train MSE =  75.69015502929688
Test MSE =  80.52273559570312
---------------------------
10/10 [==============================] - 0s 1ms/step
Train R2 =  -0.6083204643680564
Test R2 =  -0.6252191885401774
-----------------------------
Train MAE =  6.830878044902783
Test MAE =  6.928432395352082
